In [1]:
import os
os.chdir('../../')
import sys
sys.path.insert(0, './python')

In [2]:
import caffe

In [3]:
from pylab import *
%matplotlib inline

In [4]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [5]:
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
#solver = caffe.SGDSolver('examples/mlp/test/mlp_solver.prototxt')
net1 = caffe.Net('examples/mlp/test/retrain/mlp_train_test.prototxt','examples/mlp/test/test.caffemodel', caffe.TEST)

In [6]:
# during training, each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in net1.blobs.items()]

[('data', (100, 1, 28, 28)),
 ('label', (100,)),
 ('data_mnist_0_split_0', (100, 1, 28, 28)),
 ('data_mnist_0_split_1', (100, 1, 28, 28)),
 ('label_mnist_1_split_0', (100,)),
 ('label_mnist_1_split_1', (100,)),
 ('ip1', (100, 200)),
 ('ip1-1', (100, 50)),
 ('ip2', (100, 200)),
 ('ip2-1', (100, 50)),
 ('ip3', (100, 10)),
 ('ip3-1', (100, 20)),
 ('ip3_flatten', (100, 10)),
 ('concat1', (100, 30)),
 ('ip4', (100, 10)),
 ('ip4_ip4_0_split_0', (100, 10)),
 ('ip4_ip4_0_split_1', (100, 10)),
 ('accuracy', ()),
 ('loss', ())]

In [7]:
# just print the weight sizes (we'll omit the biases)
[(k, v[0].data.shape) for k, v in net1.params.items()]

[('ip1', (200, 784)),
 ('ip1-1', (50, 784)),
 ('ip2', (200, 200)),
 ('ip2-1', (50, 50)),
 ('ip3', (10, 200)),
 ('ip3-1', (20, 50)),
 ('ip4', (10, 30))]

In [ ]:
net1.forward()

In [ ]:
with open('examples/mlp/test/WB/ip1_w.txt', 'w') as file:
    for i in range(200):
        for item in net1.params['ip1'][0].data[i]:
            file.write("{}\n".format(item))
            
file.close()
with open('examples/mlp/test/WB/ip1_b.txt', 'w') as file:
    for item in net1.params['ip1'][1].data:
        file.write("{}\n".format(item))
            
file.close()

In [ ]:
with open('examples/mlp/test/WB/ip2_w.txt', 'w') as file:
    for i in range(200):
        for item in net1.params['ip2'][0].data[i]:
            file.write("{}\n".format(item))
            
file.close()
with open('examples/mlp/test/WB/ip2_b.txt', 'w') as file:
    for item in net1.params['ip2'][1].data:
        file.write("{}\n".format(item))
            
file.close()

In [ ]:
with open('examples/mlp/test/WB/ip3_w.txt', 'w') as file:
    for i in range(10):
        for item in net1.params['ip3'][0].data[i]:
            file.write("{}\n".format(item))
            
file.close()
with open('examples/mlp/test/WB/ip3_b.txt', 'w') as file:
    for item in net1.params['ip3'][1].data:
        file.write("{}\n".format(item))
            
file.close()

In [ ]:
import h5py 
with h5py.File('pruned_weights.h5', 'w') as hf:
    gw = hf.create_group('weights')
    gw.create_dataset('W1', data = net1.params['ip1'][0].data)
    gw.create_dataset('W2', data = net1.params['ip2'][0].data)
    gw.create_dataset('W3', data = net1.params['ip3'][0].data)
    
    gb = hf.create_group('bias')
    gb.create_dataset('B1', data = net1.params['ip1'][1].data)
    gb.create_dataset('B2', data = net1.params['ip2'][1].data)
    gb.create_dataset('B3', data = net1.params['ip3'][1].data)


In [ ]:
net1.params['ip1'][0].data

In [8]:
net1.save('examples/mlp/test/retrain/weights.caffemodel')